In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics.pairwise import pairwise_distances, euclidean_distances
import scipy
import sys
# Import T. Kipf's GCN implementation
# https://github.com/tkipf/keras-gcn
sys.path.append('../keras-gcn/')

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical

from kegra.layers.graph import GraphConvolution
from kegra.utils import *
%matplotlib inline

### Basic usage

In [142]:
nodes = 2500
n_features = 600

In [143]:
# Loading data
# X : (nodes, n_features) random features matrix of 0/1
X = np.matrix(np.random.choice([0,1], (nodes, n_features)))
# A : (nodes, nodes) random adjacency matrix
A = scipy.sparse.rand(nodes, nodes, density=0.4, format='csr')
A.data[:] = 1
# (nodes, 1) vector with the category of each row
y = np.random.choice([0,1], nodes).reshape((nodes, 1))
# Converts a class vector to binary class matrix.
y = to_categorical(y)

In [144]:
X.shape, A.shape

((2500, 600), (2500, 2500))

In [145]:
# split for training/validation and testing
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [146]:
# Normalize X 
X = X/X.sum(1).reshape(-1, 1)

In [147]:
# Local Pooling
# Preprocessing A: multiplication with A means that, for every node, we sum up all the feature vectors 
# of all neighboring nodes but not the node itself (unless there are self-loops in the graph). 
# We can "fix" this by enforcing self-loops in the graph: we simply add the identity matrix to A.
A_ = preprocess_adj(A, True)
support = 1
graph = [X, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [148]:
X_in = Input(shape=(X.shape[1],))

# Define model architecture
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [149]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999
NB_EPOCH = 200
PATIENCE = 10 # early stopping patience

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
"accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 0.6931 train_acc= 0.5571 val_loss= 0.6932 val_acc= 0.4667 time= 1.6878
Epoch: 0002 train_loss= 0.6931 train_acc= 0.5571 val_loss= 0.6932 val_acc= 0.4667 time= 0.8144
Epoch: 0003 train_loss= 0.6931 train_acc= 0.5571 val_loss= 0.6932 val_acc= 0.4667 time= 0.8450
Epoch: 0004 train_loss= 0.6930 train_acc= 0.5571 val_loss= 0.6932 val_acc= 0.4667 time= 0.8042
Epoch: 0005 train_loss= 0.6930 train_acc= 0.5571 val_loss= 0.6932 val_acc= 0.4667 time= 0.7844
Epoch: 0006 train_loss= 0.6929 train_acc= 0.5571 val_loss= 0.6933 val_acc= 0.4667 time= 0.7812
Epoch: 0007 train_loss= 0.6928 train_acc= 0.5571 val_loss= 0.6933 val_acc= 0.4667 time= 0.7998
Epoch: 0008 train_loss= 0.6926 train_acc= 0.5571 val_loss= 0.6935 val_acc= 0.4667 time= 0.7701
Epoch: 0009 train_loss= 0.6922 train_acc= 0.5571 val_loss= 0.6938 val_acc= 0.4667 time= 0.7850
Epoch: 0010 train_loss= 0.6917 train_acc= 0.5571 val_loss= 0.6941 val_acc= 0.4667 time= 0.7654
Epoch: 0011 train_loss= 0.6911 train_acc= 0.5571 v

In [150]:
preds

array([[ 0.51364863,  0.48635143],
       [ 0.51326466,  0.48673531],
       [ 0.51361322,  0.48638678],
       ..., 
       [ 0.51359695,  0.48640305],
       [ 0.51323998,  0.48675999],
       [ 0.51386386,  0.48613608]], dtype=float32)

### Adding missing values

If X contains randomly NaN values, what can we do to no have NaN predictions and a Nan loss ?  
We see two to face this issue :
- Fill NaN with 0.5 or the columns mean 
- Fill NaN with 0 and translate the other values (NaN->0, 0->1, 1->2) (https://www.kaggle.com/c/predict-west-nile-virus/discussion/13725#74831)

In [164]:
nodes = 2500
n_features = 600

#### First method for NaN : Translation and filling NaN with 0s

In [174]:
# We randomly add nan values to X
X = np.matrix(np.random.choice([0,1, np.nan], (nodes, n_features)))
A = scipy.sparse.rand(nodes, nodes, density=0.4, format='csr')
A.data[:] = 1
# (nodes, 1) vector with the success
y = np.random.choice([0,1], nodes).reshape((nodes, 1))
# Converts a class vector to binary class matrix.
y = to_categorical(y)

y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [152]:
# translation
X = X+1

# Normalize X 
X = X/np.nansum(X,1).reshape(-1, 1)

# filling NaN with 0s
where_are_NaNs = np.isnan(X)
X[where_are_NaNs] = 0

In [137]:
# local pool
A_ = preprocess_adj(A, True)
support = 1
graph = [X, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [138]:
X_in = Input(shape=(X.shape[1],))

# Define model architecture
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [140]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999
NB_EPOCH = 200
PATIENCE = 10 # early stopping patience

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] <= best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
"accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7760
Epoch: 0002 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7983
Epoch: 0003 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7675
Epoch: 0004 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7971
Epoch: 0005 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7507
Epoch: 0006 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6905 val_acc= 0.5433 time= 0.7613
Epoch: 0007 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6906 val_acc= 0.5433 time= 0.7634
Epoch: 0008 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6904 val_acc= 0.5433 time= 0.7934
Epoch: 0009 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6904 val_acc= 0.5433 time= 0.8393
Epoch: 0010 train_loss= 0.6923 train_acc= 0.5214 val_loss= 0.6904 val_acc= 0.5433 time= 0.8770
Epoch: 0011 train_loss= 0.6923 train_acc= 0.5214 v

In [141]:
preds

array([[ 0.47942787,  0.52057219],
       [ 0.48045942,  0.51954055],
       [ 0.47922593,  0.52077407],
       ..., 
       [ 0.47943738,  0.52056259],
       [ 0.48083887,  0.51916116],
       [ 0.48068473,  0.5193153 ]], dtype=float32)

#### Second method for NaN : filling NaN with column means

In [ ]:
# We randomly add nan values to X
X = np.matrix(np.random.choice([0,1, np.nan], (nodes, n_features)))
A = scipy.sparse.rand(nodes, nodes, density=0.4, format='csr')
A.data[:] = 1
# (nodes, 1) vector with the success
y = np.random.choice([0,1], nodes).reshape((nodes, 1))
# Converts a class vector to binary class matrix.
y = to_categorical(y)

y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [168]:
col_mean = np.nanmean(X, axis=0)
#Find indicies that you need to replace
inds = np.where(np.isnan(X))
#Place column means in the indices. Align the arrays using take
X[inds] = np.take(col_mean, inds[1])

In [170]:
# local pool
A_ = preprocess_adj(A, True)
support = 1
graph = [X, A_]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [171]:
X_in = Input(shape=(X.shape[1],))

# Define model architecture
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', W_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

# Compile model
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [172]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999
NB_EPOCH = 200
PATIENCE = 10 # early stopping patience

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] <= best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
"accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 1.3742 train_acc= 0.4857 val_loss= 1.2077 val_acc= 0.5533 time= 1.4730
Epoch: 0002 train_loss= 0.8755 train_acc= 0.4857 val_loss= 0.7973 val_acc= 0.5533 time= 0.8854
Epoch: 0003 train_loss= 0.7302 train_acc= 0.5143 val_loss= 0.7706 val_acc= 0.4467 time= 0.8353
Epoch: 0004 train_loss= 0.6931 train_acc= 0.4857 val_loss= 0.6931 val_acc= 0.5533 time= 0.7949
Epoch: 0005 train_loss= 0.6931 train_acc= 0.4857 val_loss= 0.6931 val_acc= 0.5533 time= 0.8277
Epoch: 0006 train_loss= 0.6930 train_acc= 0.5143 val_loss= 0.6937 val_acc= 0.4467 time= 0.8052
Epoch: 0007 train_loss= 0.8060 train_acc= 0.5143 val_loss= 0.8744 val_acc= 0.4467 time= 0.7911
Epoch: 0008 train_loss= 0.6934 train_acc= 0.4857 val_loss= 0.6922 val_acc= 0.5533 time= 0.8231
Epoch: 0009 train_loss= 0.6931 train_acc= 0.4857 val_loss= 0.6931 val_acc= 0.5533 time= 0.8885
Epoch: 0010 train_loss= 0.6933 train_acc= 0.4857 val_loss= 0.6925 val_acc= 0.5533 time= 0.8608
Epoch: 0011 train_loss= 0.6931 train_acc= 0.5143 v

In [173]:
preds

array([[ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       ..., 
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5]], dtype=float32)